In [1]:
import numpy as np
import math
import random

In [2]:
import escher
import escher.urls
import cobra
import cobra.test
import json
import os
from IPython.display import HTML

# Make this local
b = escher.Builder(map_json='/Users/LAURENCE/Desktop/Senior Design/RECON1.Central.json')
#b.display_in_browser(scroll_behavior='pan')

cobra/io/__init__.py:17 UserWarning: cobra.io.sbml requires libsbml


In [3]:
model = cobra.io.load_matlab_model('/Users/LAURENCE/Desktop/Senior Design/modelRecon2', 'modelRecon2')
solution = model.optimize()
print('Growth rate: %.2f' % solution.f)

Growth rate: 3.20


cobra/io/mat.py:136 FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.


In [4]:
rxn_name = 'ARGSS, ARGSL, ORNt4m, ORNt3m, OCBTm, CBPSam, CSm, PCm, ADK3, NDPK8, RNDR1'

In [5]:
v_matrix = [0]*33

In [6]:
#rxn_name = raw_input("What are the names of your reactions:   ")

rxns = rxn_name.split(', ')
print rxns

['ARGSS', 'ARGSL', 'ORNt4m', 'ORNt3m', 'OCBTm', 'CBPSam', 'CSm', 'PCm', 'ADK3', 'NDPK8', 'RNDR1']


In [7]:
v_count = 0
for c in rxns:
    v_matrix[v_count] = np.sign(model.solution.x_dict[c])
    v_count = v_count + 1

v_array = np.asarray(v_matrix)
v_diag = np.diag(v_array)
v_diag.shape
print v_diag

# v_diag is our sign(v)

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0. -1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [8]:
import csv
cr = csv.reader(open("ASS_S_matrix_3.csv","rb"))
arr = range(33) #adjust to needed
x = 0
for row in cr:    
    arr[x] = row
    x += 1

#print arr
# arr is our S for the subnetwork

In [9]:
S = np.asarray(arr)
S_num = np.array(S, np.float)

sign_v_S = v_diag.dot(S_num)
#print S_num
# sign_v_S is our S normalized to v

sign_v_S_T = np.transpose(sign_v_S)
# sign_v_S_T is our sign(v)S^T array

In [10]:
Keqs = [0.501, 0.0065, 1, 1, 120000, 1.01, 0.00000086, 5.41, 1, 0.287, 1]

In [11]:
lnKeqs = [0] * len(Keqs)
Keq_counter = 0
for d in Keqs:
    if d == 0:
        lnKeqs[Keq_counter] = 0
    else:
        lnKeqs[Keq_counter] = math.log(d)
        Keq_counter = Keq_counter + 1
        #print d
        
lnKeqs_array = np.asarray(lnKeqs)
print lnKeqs_array

[ -6.91149178e-01  -5.03595310e+00   0.00000000e+00   0.00000000e+00
   1.16952470e+01   9.95033085e-03  -1.39663334e+01   1.68824909e+00
   0.00000000e+00  -1.24827306e+00   0.00000000e+00]


In [12]:
# setting bounds
y_bounds = (-20,0)

In [13]:
c = random.uniform(-1,1)
print c
c_array = [c]*33
print c_array

0.692816428672
[0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646, 0.6928164286715646]


* v_diag is our sign(v)
* sign_v_S_T is our sign(v) S^T array
* lnKeqs_array is an array of our ln(Keqs)
* y_bounds is our bound
* c_array is an array of a random float generated between -1 and 1

In [14]:
from scipy.optimize import linprog
res = linprog(c_array, A_ub = sign_v_S_T, b_ub = lnKeqs_array, bounds= y_bounds, options={"disp": True})

Optimization failed. Unable to find a feasible starting point.
